In [ ]:
%load_ext autoreload
%autoreload 2

# import tensorflow as tf
# # Make tensorflow not take over the entire GPU memory
# for gpu in tf.config.experimental.list_physical_devices('GPU'):
#     tf.config.experimental.set_memory_growth(gpu, True)

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Video
# from tfga import GeometricAlgebra

import torch
from torch.nn.functional import pad
from torch_ga import GeometricAlgebra
# from torch_ga.blades import BladeKind
# from torch_ga.layers import GeometricProductConv1D

np.set_printoptions(precision=2, suppress=True)

# FA: still to do

In [ ]:
# !pip install ipywidgets

### Introduction

Classical electromagnetism is most often described using maxwell's equations. Instead, we can also describe it using a Lagrange density and an action which is the spacetime integral over the Lagrange density.
The field is represented by a 4-vector in the [spacetime-algebra](https://en.wikipedia.org/wiki/Spacetime_algebra) where the first component is the electric potential and the last three components are the magnetic vector potential. Such as 4-vector is given at every point in spacetime. The Lagrangian density at a spacetime point $X = (t, x, y, z)$ for such a 4-vector field $A(X)$ when speed of light $c = 1$ and without external sources is given by the following equation:

$\mathcal{L}(A, X) = \langle \nabla_X A(X) * \widetilde{\nabla_X A}(X) \rangle_0$

The [principle of stationary action](https://en.wikipedia.org/wiki/Principle_of_least_action) then states that the classical solution of the field is achieved when the action

$S(A) = \int_{X}{\mathcal{L}(A, X) dX}$

does not change anymore, that is $\delta S(A) = 0$.

### Goal
Below we will obtain an entire space-time field configuration $A(X)$ given only some boundary conditions and a function for the action given $A$. We will then use Tensorflow's optimizer to
find a field configuration that makes the action stationary.

### Create the spacetime algebra
Here we initialize a `tfga.GeometricAlgebra` instance with bases $e_0=e_t, e_1=e_x, e_2=e_y, e_3=e_z$ and corresponding metric $[-1, 1, 1, 1]$. We will use this
when calculating the action later as we need the geometric product and reversion operations.

In [ ]:
ga = GeometricAlgebra([-1, 1, 1, 1])

### Calculate the action
Now we create a function which returns the action $S$ given a field configuration $A(X)$ on a discretized spacetime lattice of size $[N, N, N, N]$. We use the following boundary conditions for $A(X)$:

$A_{t=-1} = 0, A_{t=N} = 0$

$A_{x=-1} = 10 sin(4 * \pi / N * t) e_0, A_{x=N} = -5 e_0$

$A_{y=-1} = 0, A_{y=N} = 0$

$A_{z=-1} = 0, A_{z=N} = 0$

As a reminder, $e_0$ is the electric potential part of the 4-vector, so we have a periodic sine electric potential that changes over time (two periods in total) and amplitude `10` at the lower x boundary and a constant negative electric potential of `-5` at the upper x boundary.

In [ ]:
def get_action(config_a_variable,grid_size):
    # config_a_variable will be of shape [N, N, N, N, 4].
    # The last axis' values are the e0, e1, e2, e3 parts of the multivector.

    # Finite differences in each direction using padding.
    # Example with zero padding (ie. zeros on the boundary):
    # 1 2 3
    #   1 2 3 0 padded right
    # - 0 1 2 3 padded left
    # = 1 1 1-3 padded right - padded left
    # As spacing we use 1 so we don't need to divide by anything here.
    
    # Also use the boundary conditions in the padded values here.
    # This gets a bit verbose because of the pad syntax esepcially since we only want to pad the
    # first index of the last axis with non-zeros.

    # Create time-varying boundary condition. Start with sine of shape [N].
    # Then reshape to [N, 1, N, N, 1] which we can concatenate with the
    # original values.
    pad_values = 10.0 * torch.sin(2.0 * torch.arange(grid_size[0], dtype=torch.float32) * 2.0 *  torch.tensor(np.pi, dtype=torch.float32) / grid_size[0])
    pad_values = torch.unsqueeze(pad_values, dim=-1)
    pad_values = torch.unsqueeze(pad_values, dim=-1)
    pad_values = torch.unsqueeze(pad_values, dim=-1)
    pad_values = torch.unsqueeze(pad_values, dim=-1)
    # print(f"pad_values.shape={pad_values.shape}")
    pad_values = torch.tile(pad_values, [1, 1, grid_size[0], grid_size[0], 1])
    # print(f"pad_values.shape={pad_values.shape}")
    # print(f"config_a_variable.shape={config_a_variable.shape}")

    p1 = pad(config_a_variable[..., 1:], [0, 0] + [0, 0] +  [0, 0]+  [1, 0]+  [0, 0])
    # print(f"p1.shape={p1.shape}")
    # print(f"torch.concat([pad_values, config_a_variable[..., :1]], dim=1).shape={torch.concat([pad_values, config_a_variable[..., :1]], dim=1).shape}")
    config_left_pad_x = torch.concat([
        torch.concat([pad_values, config_a_variable[..., :1]], dim=1),
        # pad(config_a_variable[..., 1:]+ [[0, 0]+ [1, 0]+ [0, 0]+ [0, 0]+ [0, 0]]),
        p1,
    ], dim=-1)

    config_right_pad_x = torch.concat([
        pad(config_a_variable[..., :1], [0, 0]+ [0, 0]+ [0, 0]+ [0, 1]+ [0, 0], "constant", -5),
        pad(config_a_variable[..., 1:], [0, 0]+ [0, 0]+[0, 0]+ [0, 1]+ [0, 0]),
    ], axis=-1)

    config_left_pad_y = torch.concat([
        pad(config_a_variable[..., :1], [0, 0]+ [0, 0]+ [1, 0]+ [0, 0]+ [0, 0]),
        pad(config_a_variable[..., 1:], [0, 0]+ [0, 0]+ [1, 0]+ [0, 0]+ [0, 0]),
    ], axis=-1)

    config_dt_a = (
        pad(config_a_variable, [0, 0]+ [0, 0]+ [0, 0]+ [0, 0]+ [0, 1]) -
        pad(config_a_variable, [0, 0]+ [0, 0]+ [0, 0]+ [0, 0]+ [1, 0])
    )
    config_dx_a = config_right_pad_x - config_left_pad_x
    config_dy_a = (
        pad(config_a_variable, [0, 0]+ [0, 0]+ [0, 1]+ [0, 0]+ [0, 0]) -
        config_left_pad_y
    )
    config_dz_a = (
        pad(config_a_variable, [0, 0]+ [0, 1]+ [0, 0]+ [0, 0]+ [0, 0]) -
        pad(config_a_variable, [0, 0]+ [1, 0]+ [0, 0]+ [0, 0]+ [0, 0])
    )

    # Convert to multivectors so we can use GA ops we need in the Lagrangian:
    # the geometric product and reversion.
    config_dt_a = ga.from_tensor_with_kind(config_dt_a, "vector")
    config_dx_a = ga.from_tensor_with_kind(config_dx_a, "vector")
    config_dy_a = ga.from_tensor_with_kind(config_dy_a, "vector")
    config_dz_a = ga.from_tensor_with_kind(config_dz_a, "vector")

    # Sum all the derivatives according to the action / Lagrangian and return a single scalar value
    return torch.mean(ga.geom_prod(config_dt_a, ga.reversion(config_dt_a))[..., 0]).abs()/4 + \
        torch.mean(ga.geom_prod(config_dx_a, ga.reversion(config_dx_a))[..., 0]).abs()/4 + \
        torch.mean(ga.geom_prod(config_dy_a, ga.reversion(config_dy_a))[..., 0]).abs()/4 + \
        torch.mean(ga.geom_prod(config_dz_a, ga.reversion(config_dz_a))[..., 0]).abs()/4

### Initialize the 4-vector field variable randomly

In [ ]:
grid_size = [16, 16, 16, 16]
# grid_size = [4]*4
# grid_size = [8]*4
# grid_size = [6]*4

torch.manual_seed(0)
config_a_variable = torch.tensor(torch.randn([*grid_size, 4]),requires_grad=True)

In [ ]:
config_a_variable.shape

### Optimize the 4-vector field variable to make the action stationary
In order to make the action stationary we use a loss function that is minimal when the action is stationary (ie. the gradient of the action with respect to the field configuration is 0).
We use the mean-squared error to create such a loss function, although other functions such as the absolute value would work too.

We use Tensorflow's Adam optimizer to find a field configuration which minimizes the loss.

In [ ]:
# optimizer = torch.optim.Adam(lr=0.01)

# @tf.function
# def train_step(config_a_variable):
#     # Principle of stationary action:
#     # Minimize the distance of gradient of the action to zero with respect to our field
#     with tf.GradientTape() as tape_outer:
#         tape_outer.watch(config_a_variable)
#         with tf.GradientTape() as tape:
#             tape.watch(config_a_variable)
#             loss = get_action(config_a_variable)

#         grads = tape.gradient(loss, [config_a_variable])
#         grads_mse = tf.reduce_mean(tf.square(grads))
#     grads2 = tape_outer.gradient(grads_mse, [config_a_variable])
#     optimizer.apply_gradients(zip(grads2, [config_a_variable]))

# optimizer = torch.optim.Adam(lr=0.01)
# torch.optim.SGD(self.module.parameters(), lr=2e-2)
# 
from torch.autograd import grad 
from tqdm.notebook import tqdm
# def train_step(config_a_variable):
#     # Principle of stationary action:
#     # Minimize the distance of gradient of the action to zero with respect to our field
    
#     # config_a_variable.requires_grad=True
    
#     # optimizer.zero_grad()
#     lr = 1e-2
    
#     loss = get_action(config_a_variable,grid_size)
#     print(f"loss={loss}")

#     grads = grad(loss, [config_a_variable], create_graph=True)[0]
#     grads_mse = torch.mean(torch.square(grads))
        
#     grads2 = grad(grads_mse, [config_a_variable])[0]
#     # optimizer.apply_gradients(zip(grads2, [config_a_variable]))
#     config_a_variable = config_a_variable - lr * grads2


for ki in  (pbar := tqdm(range(3000), miniters=int(223265/100))):
    # train_step(config_a_variable)
    # Principle of stationary action:
    # Minimize the distance of gradient of the action to zero with respect to our field
    
    # config_a_variable.requires_grad=True
    
    # optimizer.zero_grad()
    # lr = 2e-2
    # lr = 7e-2
    # lr = 1e-2
    # lr = 3e-3
    # lr = 7.
    lr = 14.
    
    loss = get_action(config_a_variable,grid_size)
    if ki%500==0: print(f"loss({ki})={loss}")
    pbar.set_description(f"loss({ki:04})={loss:0.6f}")

    grads = grad(loss, [config_a_variable], create_graph=True)[0]
    grads_mse = torch.mean(torch.square(grads))
        
    grads2 = grad(grads_mse, [config_a_variable])[0]
    # optimizer.apply_gradients(zip(grads2, [config_a_variable]))
    # config_a_variable = config_a_variable - lr * grads2   +lr/10*grads
    config_a_variable = config_a_variable - lr * grads2/(1e-12 + (grads2**2).sum()**.5 )    

In [ ]:
grads2, grad

# config_a_variable = config_a_variable.cuda()
# config_a_variable.requires_grad_=True

### Extract and visualize the optimized electric field
Now we can take the result, that is the $A$ at every spacetime point and visualize it. Obviously we can't visualize a 4 dimensional 4-vector field. However we can look at
individual 2D slices of the electric potential field, which is the first component of the 4-vector, where the other two coordinates take on a specific value.

In [ ]:
# Plot electric potential slices. We are not plotting the boundaries here.

config_a_variable = config_a_variable.detach().cpu()
plt.figure(figsize=(7, 7))
plt.imshow(config_a_variable[..., 0, 0, 0])
plt.colorbar()
plt.title("Electric potential in TX plane Y=0, Z=0")
plt.xlabel("X")
plt.ylabel("T")
plt.show()

plt.figure(figsize=(7, 7))
plt.imshow(config_a_variable[..., 0, 5, :, :, 0].detach().cpu())
plt.colorbar()
plt.title("Electric potential in YZ plane T=0, X=5")
plt.xlabel("Z")
plt.ylabel("Y")
plt.show()

plt.figure(figsize=(7, 7))
plt.imshow(config_a_variable[..., 2, :, :, 0, 0].detach().cpu())
plt.colorbar()
plt.title("Electric potential in XY plane T=2, Z=0")
plt.xlabel("Y")
plt.ylabel("X")
plt.show()

In the first figure we can see the potential close to X=0 (where we applied the sine boundary condition) changing over time.
The second figure shows the YZ slice at T=0, X=5 where the potential is almost constant but we still have a radial symmetry.
The last figure shows the XY slice at T=2, Z=0 where the potential takes its maximum value around X=0 if we look at the first figure. We can also see that on upper boundary of X that we have a negative potential as we applied a constant negative electric potential for boundary condition there.

We can also visualize the XY slices over time in a video. For this I saved the XY slices at all times and converted them to a webm using ffmpeg. Here we can see the electric potential close to X=0 changing over time as we expected from the boundary condition. (Direct link: [em_output/electric_potential.webm](https://raw.githubusercontent.com/RobinKa/tfga/master/notebooks/em_output/electric_potential.webm))

In [ ]:
Video("./em_output/electric_potential.webm")

Next we can look at the electric vector field corresponding to the electric potential: $E = -\nabla_{x,y,z} \langle A(X) \rangle_{e0} - \nabla_t \langle A(X) \rangle_{e1,e2,e3}$

In [ ]:
def draw_electric_field_xy(t, z):
    # Extract XY slice of electric potential [T=t, X, Y, Z=0, 0]
    electric_potential = config_a_variable[t, :, :, z, 0]
    magnetic_potential_t = config_a_variable[t, :, :, z, 1:]
    magnetic_potential_t2 = config_a_variable[t+1, :, :, z, 1:]

    # The electric field can be obtained from the 4-vector potential:
    # E = - (d/dx, d/dy, d/dz) <A>_e0 - d/dt <A>_e1,e2,e3
    # We can use finite differences again to approximate the derivatives.
    # We also need to get rid of the last element of the respective other axis,
    # since we couldn't calculate the last finite difference as that would
    # require using the boundary condition (which is possible, but would require extra code).

    # Start with -(d/dx, d/dy, d/dz) <A>_e0
    ex = -(electric_potential[1:, :-1] - electric_potential[:-1, :-1])
    ey = -(electric_potential[:-1, 1:] - electric_potential[:-1, :-1])
    
    # Calculate d/dt <A>_e1,e2,e3 and add it to the previous calculation
    dt_mag_a = -(magnetic_potential_t2[-1, :-1] - magnetic_potential_t[:-1, :-1])

    ex += dt_mag_a[..., 0]
    ey += dt_mag_a[..., 1]

    ys, xs = np.meshgrid(np.arange(ex.shape[0]), np.arange(ex.shape[1]))

    plt.figure(figsize=(10, 10))
    plt.quiver(ys, xs, ey, ex, scale=10, scale_units="inches")
    plt.xlabel("Y")
    plt.ylabel("X")
    plt.title("Electric field XY at T=%d, Z=%d" % (t, z))

draw_electric_field_xy(t=2, z=0)
plt.show()

And again I made a video showing all the time slices. (Direct link: [em_output/electric_field.webm](https://raw.githubusercontent.com/RobinKa/tfga/master/notebooks/em_output/electric_field.webm))

In [ ]:
# Video("./em_output/electric_field.webm")

Video("notebooks_em_output_electric_field.webm")
